In [0]:
import datetime
from xgboost import XGBRegressor
import xlsxwriter
import chardet
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import array
from numpy import argmax

In [0]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 6.8MB/s 


In [0]:

input_data = pd.read_excel('X_train.xlsx')
input_data = input_data.iloc[:, :-1].values
print(input_data)


[[ 0.00000e+00  2.10000e+01  1.20000e+02  7.50000e+00  2.86000e-01
   8.10000e-01  3.89407e+02  2.34600e+01  7.78400e+02  1.35000e+00
   2.75000e+00  1.47200e+01  1.67700e+01  8.28000e+00  8.00000e+00
  -2.18400e+02 -1.17800e+02 -5.82000e+01 -1.04000e+02 -1.16900e+02
  -5.30000e+01  6.92000e+01  1.85000e+01 -9.85000e+01 -1.18100e+02
  -1.23000e+02]
 [ 1.00000e+00  1.85000e+01  1.35000e+02  7.20000e+00  2.68000e-01
   7.50000e-01  3.60685e+02  9.90000e+00  5.54064e+02  6.00000e-01
   3.11000e+00  1.53200e+01  1.32700e+01  1.65600e+01  8.00000e+00
  -6.84000e+01 -4.28000e+01  1.68000e+01 -2.90000e+01 -4.19000e+01
   2.20000e+01  1.44200e+02  9.35000e+01 -2.35000e+01 -4.31000e+01
  -4.80000e+01]
 [ 2.00000e+00  2.00000e+01  1.60000e+02  7.40000e+00  1.99000e-01
   8.55000e-01  4.10949e+02  5.54000e+00  1.65536e+02  6.00000e-01
   5.00000e+00  2.44900e+01  2.63900e+01  6.21000e+00  8.00000e+00
  -4.34000e+01 -3.03000e+01  2.93000e+01 -1.65000e+01 -2.94000e+01
   3.45000e+01  1.56700e+02  1

In [0]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(output_data)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[11 16  6 13 11 14  5 15 11 16  9 13 16  7 10  6  3  5  7  8  3  1  2  8
  1 12  2  0  9  4  3 14  5 15]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
xgb_model = XGBRegressor(max_depth=8, 
                         n_estimators=500, 
                         min_child_weight=1000,  
                         colsample_bytree=0.7, 
                         subsample=0.7, 
                         eta=0.3, 
                         seed=0)
xgb_model.fit(input_data, 
              onehot_encoded , 
              eval_metric="rmse", 
              eval_set=[(input_data, onehot_encoded )], 
              verbose=20, 
              )

[04:22:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.466264
[20]	validation_0-rmse:0.24338
[40]	validation_0-rmse:0.235789
[60]	validation_0-rmse:0.235355
[80]	validation_0-rmse:0.235307
[100]	validation_0-rmse:0.235339
[120]	validation_0-rmse:0.235296
[140]	validation_0-rmse:0.235324
[160]	validation_0-rmse:0.235321
[180]	validation_0-rmse:0.235319
[200]	validation_0-rmse:0.235385
[220]	validation_0-rmse:0.235296
[240]	validation_0-rmse:0.235294
[260]	validation_0-rmse:0.235294
[280]	validation_0-rmse:0.235321
[300]	validation_0-rmse:0.235347
[320]	validation_0-rmse:0.235302
[340]	validation_0-rmse:0.235645
[360]	validation_0-rmse:0.235468
[380]	validation_0-rmse:0.235642
[400]	validation_0-rmse:0.235297
[420]	validation_0-rmse:0.235296
[440]	validation_0-rmse:0.2353
[460]	validation_0-rmse:0.235296
[480]	validation_0-rmse:0.235379
[499]	validation_0-rmse:0.235393


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=1000, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=None,
             subsample=0.7, verbosity=1)

In [0]:
prediction = xgb_model.predict(input_data)

In [53]:
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(input_data,onehot_encoded)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [68]:
rf_train_pred = rf_model.predict(input_data)
print(rf_train_pred)

[[0.00222222 0.01613636 0.01617605 0.         0.00894444 0.0065
  0.01535714 0.0126912  0.00555556 0.0564531  0.01270707 0.77966017
  0.00615079 0.0353824  0.01463492 0.00777778 0.00365079]
 [0.00768926 0.02114164 0.01510073 0.00911783 0.01507937 0.005
  0.01123016 0.01150794 0.01292735 0.05025946 0.0190293  0.00730159
  0.0180464  0.01507937 0.03623016 0.01480159 0.73045788]
 [0.00978716 0.04114502 0.04170924 0.00767677 0.01893001 0.01203247
  0.46175108 0.04893939 0.02481169 0.08518398 0.01645382 0.03202381
  0.01046898 0.01642857 0.08251804 0.02256061 0.06757937]
 [0.01717532 0.03275613 0.02814574 0.01121573 0.0399531  0.00793651
  0.01768398 0.02410534 0.02968254 0.04379509 0.02275613 0.01982684
  0.02812771 0.50948773 0.10106421 0.04505772 0.02123016]
 [0.00222222 0.01977273 0.01799423 0.         0.00894444 0.0065
  0.01535714 0.01632756 0.00919192 0.0564531  0.01452525 0.74147835
  0.00615079 0.0153824  0.05463492 0.00959596 0.00546898]
 [0.01786797 0.03121934 0.03561688 0.021518

In [57]:
print('Train rmse:', np.sqrt(mean_squared_error(onehot_encoded, rf_train_pred)))

Train rmse: 0.1273710198239808
